In [1]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import seaborn as sns
from datetime import date

import warnings
warnings.filterwarnings("ignore")

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Environment settings: 
pd.set_option('display.max_column', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_colwidth', 500)
pd.set_option('expand_frame_repr', True)

In [2]:
def get_page(url):
    """Download a webpage and return a beautiful soup doc"""
    ##### Web scrapper for infinite scrolling page #####
    driver = webdriver.Chrome(executable_path=r"E:\Chromedriver\chromedriver_win32_chrome83\chromedriver.exe")
    driver.get(url)
    time.sleep(2)  # Allow 2 seconds for the web page to open
    scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 1

    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(scroll_pause_time)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if i == 10:
            break 

    ##### Extract Reddit URLs #####
    soup = BeautifulSoup(driver.page_source, "html.parser")
    return soup

In [3]:
def get_news_tags(doc):
    """Get the list of tags containing news information"""
    news_class = "Ov(h) Pend(44px) Pstart(25px)" ## class name of div tag 
    news_list  = doc.find_all('div', {'class': news_class})
    return news_list

In [4]:
BASE_URL = 'https://finance.yahoo.com' #Global Variable 

def parse_news(news_tag):
    """Get the news data point and return dictionary"""
    news_source = news_tag.find_all('span')[0].text #source
    news_time = news_tag.find_all('span')[1].text #time
    news_headline = news_tag.find('a').text #heading
    news_url = news_tag.find('a')['href'] #link
    news_content = news_tag.find('p').text #content
    news_image = news_tag.findParent().find('img')['src'] #thumb image
    return { 'source' : news_source,
            'time' : news_time,    
            'headline' : news_headline,
            'url' : BASE_URL + news_url,
            'content' : news_content,
           }

In [5]:
def scrape_yahoo_news(url, path=None):
    """Get the yahoo finance market news and write them to CSV file """
    if path is None:
        path = 'stock-market-news.csv'
        
    #print('Requesting html page')
    doc = get_page(url)

    #print('Extracting news tags')
    news_list = get_news_tags(doc)

    #print('Parsing news tags')
    news_data = [parse_news(news_tag) for news_tag in news_list]

    #print('Save the data to a CSV')
    news_df = pd.DataFrame(news_data)
    #news_df.to_csv(path, index=None)
    
    #This return statement is optional, we are doing this just analyze the final output 
    return news_df 

In [6]:
# url = 'https://finance.yahoo.com/topic/stock-market-news/'
# doc = get_page(url)
# news_list = get_news_tags(doc)
# news_list

## Collecting Stock Market News From Yahoo Finance

In [7]:
# from datetime import datetime

# now = datetime.now()
# current_time = now.strftime("%H:%M:%S")
# print("Report Date = ", date.today())
# print("Report Time =", current_time)
# YAHOO_NEWS_URL = BASE_URL + '/topic/stock-market-news/'
# news_df = scrape_yahoo_news(YAHOO_NEWS_URL)

In [8]:
# from transformers import pipeline

### Sentiment Analysis of Yahoo Finance

In [9]:
# sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# sentiment = [None] * len(news_df)
# sentiment_score = [None] * len(news_df)
# index = -1
# for sentence in news_df['headline']:
#     index+=1
#     result = sentiment_pipeline(sentence[:512])[0]
#     sentiment[index] = result['label']
#     sentiment_score[index] = result['score']
# news_df['Sentiment'] = sentiment
# news_df['Score'] = sentiment_score


# sentiment = [None] * len(news_df)
# sentiment_score = [None] * len(news_df)
# index = -1
# for sentence in news_df['content']:
#     index+=1
#     result = sentiment_pipeline(sentence[:512])[0]
#     sentiment[index] = result['label']
#     sentiment_score[index] = result['score']
# news_df['Sentiment2'] = sentiment
# news_df['Score2'] = sentiment_score

In [10]:
# news_df[news_df['Score'] > 0.8]

### Sentiment of Headline

In [11]:
# sns.histplot(x = 'Sentiment', data = news_df[news_df['Score'] > 0.8], hue = 'Sentiment', hue_order=['POSITIVE', 'NEGATIVE']).set_title("Sentiment of Headline")

### Sentiment of Content

In [12]:
# sns.histplot(x = 'Sentiment2', data = news_df[news_df['Score2'] > 0.8], hue = 'Sentiment2', hue_order=['POSITIVE', 'NEGATIVE']).set_title("Sentiment of Content")

In [13]:
import sys
sys.path.insert(1, 'C:/Users/Woon/Desktop/Columbia/Applied Analytics/Term3/Sentiment_SNP')

import helper_functions
from helper_functions import *

In [14]:
from collections import defaultdict
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)


## Each Articles - From Finviz

In [15]:
from urllib.request import Request, urlopen
from urllib.error import HTTPError
import socket
import urllib

In [16]:
from datetime import datetime

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Report Date = ", date.today())
print("Report Time =", current_time)

Report Date =  2022-12-18
Report Time = 16:36:50


In [17]:
# Scraping useful information

url = "https://finviz.com/news.ashx"


headers = {"User-Agent": "Mozilla/5.0"}
cookies = {"CONSENT": "YES+cb.20210720-07-p0.en+FX+410"}

req = Request(url, headers=headers)

try:
    contents = urlopen(req).read() 
    soup = BeautifulSoup(contents, features="html.parser")

    link2 = []
    headline2 = []
    date2 = []

    for row in soup.find_all('tr', class_ = 'nn'):
        headline = row.find('a', class_ = 'nn-tab-link')
       
        if headline == None:
            continue

        headline2.append(headline.text)
        link = row.find('a', class_ = 'nn-tab-link')
        link2.append(link['href'])
        date = row.find('td', class_ = 'nn-date')
        date2.append(date)

except urllib.error.HTTPError as err:
    print(err.code)

except socket.timeout as se:
    print("socket timeout")

In [18]:
#Turning the data into a dataframe
df = pd.DataFrame(date2)

In [19]:
df['Date'] = df[0]
df['Headline'] = headline2
df = pd.DataFrame(list(zip(df['Date'], df['Headline'])), columns=['Date', 'Headline'])
df['Links'] = link2

In [20]:
# # Filtering only links starting with https
# df['Links_True'] = list(
#     map(lambda x: x.startswith('http'), df['Links']))

# df = df[df['Links_True'] == True].reset_index(drop=True)

In [21]:
# Creating word count of headline (no longer needed)
i = []
for string in df['Headline']:
    i.append(count_words(string))
df['word_count'] = i

In [22]:
from transformers import pipeline

In [23]:
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

In [24]:
# Sentiment analysis of headline
df['Sentiment_Head'] = -2

sentiment = [None] * len(df)
sentiment_score = [None] * len(df)
index = -1
for sentence in df['Headline']:
    index+=1
    result = sentiment_pipeline(sentence[:512])[0]
    sentiment[index] = result['label']
    sentiment_score[index] = result['score']
df['Sentiment_Head'] = sentiment
df['Score_Head'] = sentiment_score

In [25]:
# Collect entire articles

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
cookies = {"CONSENT": "YES+cb.20210720-07-p0.en+FX+410"}

article = []

for url in df['Links']:
    req = Request(url, headers=headers)

    try:
        contents = urlopen(req).read() 
        soup = BeautifulSoup(contents, features="html.parser")

        p = soup.find_all('p')
        paragraphs = []
        for x in p:
            paragraphs.append(str(x.text))

        paragraphs = ''.join(paragraphs)

        article.append(paragraphs) 

    except urllib.error.HTTPError as err:
        print(err.code)

    except socket.timeout as se:
        print("socket timeout")

df['article'] = article


In [26]:
df = df[df['Score_Head'] > 0.9]

In [27]:
# Summary of Articles
summary_pipeline = pipeline("summarization", model="pszemraj/long-t5-tglobal-base-16384-book-summary")

summary = [None] * len(df)
index = -1
for articles in df['article']:
    index+=1
    result = summary_pipeline(articles[:512])[0]
    summary[index] = result
df['summary'] = summary

Your max_length is set to 512, but you input_length is only 109. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=54)
Your max_length is set to 512, but you input_length is only 114. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=57)
Your max_length is set to 512, but you input_length is only 73. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=36)
Your max_length is set to 512, but you input_length is only 73. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=36)
Your max_length is set to 512, but you input_length is only 115. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=57)
Your max_length is set to 512, but you input_length is only 116. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=58)
Your max_length is set to 512, but you input_length is only 73. You might cons

In [28]:
# Sentiment analysis of Article Summary
df['Sentiment_Article'] = -2

sentiment = [None] * len(df)
sentiment_score = [None] * len(df)
index = -1
for sentence in df['Headline']:
    index+=1
    result = sentiment_pipeline(sentence[:512])[0]
    sentiment[index] = result['label']
    sentiment_score[index] = result['score']
df['Sentiment_Article'] = sentiment
df['Score_Article'] = sentiment_score

In [33]:
#df['summary'] = df['summary'].to_string()
#df.drop_duplicates(subset='summary', keep="last")
df.head()

,Date,Headline,Links,word_count,Sentiment_Head,Score_Head,article,summary,Sentiment_Article,Score_Article
0,04:15PM,Twitter to stop users linking to big social media platforms,https://www.bbc.co.uk/news/business-your-money-64021412?at_medium=RSS&at_campaign=KARANGA,10,NEGATIVE,0.998175,"Twitter says it will shut down accounts solely designed to promote other big social media platforms, or which link off to or contain usernames from them.The move affects content from platforms including Facebook, Instagram, Mastodon, Truth Social, Tribel, Nostr and Post, the company said. But cross-content posting from other sites will still be allowed.It is the latest of many changes since the billionaire Elon Musk bought Twitter.On Saturday, Washington Post reporter Taylor Lorenz was susp...","0 0 0 {'summary_text': 'The company says it's going to shut down accounts that aren't really designed to promote itself. Instead, it'll shut down all of those accounts and turn them into ""social media marketplaces."" That means you can no longer post from other social media sites like Facebook, Twitter, Instagram, mastodon, truth social, tribel, nostr or Post - just use your own name.'}\n1 ...",NEGATIVE,0.998175
1,04:11PM,"Stock market worries, crypto fallout, and doubts of a Santa Claus rally top week ahead",https://foxbusiness.com/markets/stock-market-worries-crypto-fallout-doubts-santa-claus-rally-top-week-ahead,15,NEGATIVE,0.997043,"\n Quotes displayed in real-time or delayed by at least 15 minutes. Market data provided by Factset.\n Powered and implemented by FactSet Digital Solutions. \n Legal Statement. Mutual Fund and ETF data provided by Refinitiv Lipper.\n This material may not be published, broadcast, rewritten, or redistributed. ©2022 FOX News Network, LLC. All rights reserved. FAQ - New Privacy PolicyKPMG chief economist Diane Swonk and The Cow Guy Group founder Scott S...","0 0 0 {'summary_text': 'The company says it's going to shut down accounts that aren't really designed to promote itself. Instead, it'll shut down all of those accounts and turn them into ""social media marketplaces."" That means you can no longer post from other social media sites like Facebook, Twitter, Instagram, mastodon, truth social, tribel, nostr or Post - just use your own name.'}\n1 ...",NEGATIVE,0.997043
2,04:04PM,Yen Jumps on Report of Kishida Flexibility on Monetary Policy,https://www.bloomberg.com/news/articles/2022-12-18/yen-jumps-on-report-of-kishida-flexibility-on-monetary-policy?srnd=markets-vp,10,NEGATIVE,0.943968,"To continue, please click the box below to let us know you're not a robot.Please make sure your browser supports JavaScript and cookies and that you are not\n blocking them from loading.\n For more information you can review our Terms of\n Service and Cookie Policy.For inquiries related to this message please contact\n our support team and provide the reference ID below.","0 0 0 {'summary_text': 'The company says it's going to shut down accounts that aren't really designed to promote itself. Instead, it'll shut down all of those accounts and turn them into ""social media marketplaces."" That means you can no longer post from other social media sites like Facebook, Twitter, Instagram, mastodon, truth social, tribel, nostr or Post - just use your own name.'}\n1 ...",NEGATIVE,0.943968
4,04:00PM,Why Marcos’s Philippine Wealth Fund Plan Is Controversial: Q&A,https://www.bloomberg.com/news/articles/2022-12-18/why-marcos-s-philippine-wealth-fund-plan-is-controversial-q-a?srnd=markets-vp,9,NEGATIVE,0.988577,"To continue, please click the box below to let us know you're not a robot.Please make sure your browser supports JavaScript and cookies and that you are not\n blocking them from loading.\n For more information you can review our Terms of\n Service and Cookie Policy.For inquiries related to this message please contact\n our support team and provide the reference ID below.","0 0 0 {'summary_text': 'The company says it's going to shut down accounts

In [31]:
## Get the bar chart from 5 rated reviews ##
freq_dict = defaultdict(int)
for sent in df["article"]:
    for word in generate_ngrams(sent,2):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace0 = horizontal_bar_chart(fd_sorted.head(10), 'green')

# Creating two subplots
fig = tools.make_subplots(rows=1, cols=1, vertical_spacing=0.04,
                          subplot_titles=["Frequent words"])
fig.append_trace(trace0, 1, 1)
#fig.append_trace(trace1, 1, 2)
fig['layout'].update(height=600, width=600, paper_bgcolor='rgb(233,233,233)', title="Word Count Plots")
py.iplot(fig, filename='word-plots');

c:\Users\Woon\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\tools.py:460: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead

